Detección de caras con webcam

In [14]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors
import math
import numpy as np

def rotate_from_origin(xy, radians):
    """Only rotate a point around the origin (0, 0)."""
    x, y = xy
    xx = x * math.cos(radians) + y * math.sin(radians)
    yy = -x * math.sin(radians) + y * math.cos(radians)

    return xx, yy

En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [31]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

faceD = FDet.FaceDetectors[1]
eyesD = FDet.EyeDetectors[1]


compensation_x_ang = 5
# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

debug = False

#Set camera resolution
cap.set(3,640)
cap.set(4,480)

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, faceD, eyesD)
    if values is not None:
        face, eyes, shape = values

        # Ángulo positivo = girar en sentido antihorario
        # x = mirar abajo, y = mirar izquierda, z = girar derecha
        # Unidad = radianes
        x_ang = y_ang = z_ang = 0   



        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Show detected facial elements
                if debug:
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                    cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                    cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)

                    for i in (33, 2, 14):
                        cv2.circle(frame, (shape[i,0], shape[i,1]), 2, (255, 0, 0), -1)

                
                ab = (shape[14]-shape[2])
                ap = (shape[33]-shape[2])
                distCheeks = np.linalg.norm(ab)
                distNose = np.linalg.norm(np.dot(ap, ab) / np.dot(ab, ab) * ab)

                z_ang = math.atan((shape[26,1] - shape[17,1]) / (shape[26,0] - shape[17,0]))
                y_ang = prop = distNose / distCheeks - 0.5
                x_ang = rotate_from_origin((lex, ley), z_ang)[1]-rotate_from_origin(shape[0], z_ang)[1] - compensation_x_ang
                x_ang /= 60

                # TRABAJAR AQUÍ



    #if debug:
    #    print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, faceD, (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, eyesD, (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0
-2.259622900879126
-2.878879142498227
-3.7602081231535385
-3.9619919223410025
-3.8364383042468777
-3.682018079364951
-5.3563736657063625
-5.927543499876567
-3.022788815968226
-3.2623897985088335
-3.2193836235852005
-2.884158723370512
-1.2731593228346085
-3.9806138724776083
-2.995690498656539
-2.5663429383659504
-3.821569608923891
1.28895173423663
-0.934215490825512
-0.47816939525387003
-0.4090285409742683
-0.9587728649573966
-0.977737542815721
-1.5268337114407893
-1.084906457395623
-1.3816691887774124
-1.9044019663301923
-1.555822408196659
-3.1387816042978245
-5.650797936097105
-8.055639867764352
-10.231640197511261
-10.713728306042238
-13.55809435915931
-11.4807501976933
-11.013686550042175
-14.831121575649322
-13.72800012232433
-11.490883232661721
-11.813311266954654
-10.281904960314387
-12.648416767467882
-10.89860716760283
-11.131379339198425
-10.567733120902039
-10.108516519427699
-9.267770157223735
-9.13389207632827
-9.442156094280046
-10.555733118041942
-10.883347478484